**Importing Libraries**

In [242]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml.html as lh
import requests

**Scraping Wikipage**

In [243]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

**Transforming data from table into DataFrame**

In [244]:
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    i=0
    for t in T.iterchildren():
        data = t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

1 Postcode
2 Borough
3 Neighbourhood



IndexError: list index out of range

In [245]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

ValueError: arrays must all be same length

In [246]:
df.head()

,Neighbourhood,PostalCode,Borough
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,Harbourfront,M5A,Downtown Toronto
3,Regent Park,M5A,Downtown Toronto
4,Lawrence Heights,M6A,North York


**Modifying column order**

In [247]:
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]

In [248]:
df.head()

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


**Renaming columns**

In [249]:
df.rename(index = str, columns = {"Postcode" :"PostalCode", "Neighbourhood\n" : "Neighbourhood"}, inplace = True)

In [250]:
df.head()

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


**Removing Not Assigned**

In [251]:
df.drop(df.loc[df['Borough']=="Not assigned"].index, inplace=True)

In [252]:
df.reset_index(drop=True, inplace=True)

In [253]:
df.head()

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


**Removing \n in Neighbourhood**

In [254]:
i = 0
temp_list = []
for index, row in df.iterrows():
    temp_list.append(df['Neighbourhood'][i].replace('\n','',True))
    i+=1  

In [255]:
len(temp_list)

211

In [256]:
df['Neighbourhood'] = temp_list

In [257]:
df.head()

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


**Grouping by Neighbourhood**

In [258]:
df1 = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [259]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [260]:
df1.shape

(103, 3)